## Importing the nn library and applying to a tensor.

In [33]:
import torch
import torch.nn as nn
from torch import Tensor
from torch import optim
import numpy as np

In [5]:
linear_model = nn.Linear(1, 1)
linear_model

Linear(in_features=1, out_features=1, bias=True)

In [17]:
y = Tensor([1,2,3]).unsqueeze(1)
print(y.shape)
linear_model(y)

torch.Size([3, 1])


tensor([[-0.4579],
        [-0.5272],
        [-0.5964]], grad_fn=<AddmmBackward>)

# Building a training loop for a linear model

## The motivation for this is to be aware that torch isn't just neural nets; it's general purpose backprop and autograd.

## First we'll generate some data                                         

In [57]:
n_samples = 5000
n_val = int(0.2 * n_samples)
x = np.random.rand(n_samples)
y = 3*x + 4 + (np.random.rand(n_samples) - 0.5) # y ~ 3x + 4 with uniform noise

shuffledIndices = torch.randperm(n_samples)
trainIndices = shuffledIndices[:-n_val]
valIndices = shuffledIndices[-n_val:]

x_train, y_train = x[trainIndices], y[trainIndices]
x_val, y_val = x[valIndices], y[valIndices]
data = [x_train, x_val, y_train, y_val] # collecting in a single var to unpack later.
data = [torch.from_numpy(d).float().unsqueeze(1) for d in data] # put data into clean form for Torch

## Next our training loop definiton

In [65]:
def training_loop(n_epochs, optimizer, model, loss, x_train, x_val, y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        pred_train = model(x_train)
        loss_train = loss(pred_train, y_train)
        
        pred_val   = model(x_val)
        loss_val   = loss(pred_val, y_val)
        
        optimizer.zero_grad() # zero out accumulated gradient
        loss_train.backward() # backprop step
        optimizer.step() # gradient descent
        
        if epoch == 1 or epoch % 500 == 0:
            print(f" Epoch: {epoch}, Training/Validation Losses {loss_train}/{loss_val}")

## Model run

In [66]:
linear_model = nn.Linear(1, 1) # should be sufficient for our linearly generated data
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2) # SGD optimizer

training_loop(3000,
              optimizer,
              linear_model,
              nn.MSELoss(),
              *data
             )

print(linear_model.weight)
print(linear_model.bias)

 Epoch: 1, Training/Validation Losses 22.277938842773438/21.952390670776367
 Epoch: 500, Training/Validation Losses 0.09619652479887009/0.0953688994050026
 Epoch: 1000, Training/Validation Losses 0.08639254420995712/0.086032435297966
 Epoch: 1500, Training/Validation Losses 0.08372040092945099/0.08355458080768585
 Epoch: 2000, Training/Validation Losses 0.08299215883016586/0.08291421830654144
 Epoch: 2500, Training/Validation Losses 0.08279364556074142/0.08275792002677917
 Epoch: 3000, Training/Validation Losses 0.08273955434560776/0.08272486180067062
Parameter containing:
tensor([[2.9905]], requires_grad=True)
Parameter containing:
tensor([4.0040], requires_grad=True)
